In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta

import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
2024-10-16 13:25:54,872 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-10-14


In [4]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-10-15


In [6]:
feature_view = fs.get_feature_view('amd_stock_fv', 12)
feature_view.init_batch_scoring(training_dataset_version=1)

In [7]:
df = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.77s) 


In [8]:
df.head()

,date,f_1__open
0,2024-10-15 00:00:00+00:00,163.21


In [9]:
import joblib

the_model = mr.get_model("ProphetModel", version=8)
model_dir = the_model.download()

model = joblib.load(model_dir + "/Prophet_model.pkl")

2024-10-16 13:27:14,588 WARNING: FutureIncompatibilityWarning: 

This is a future version incompatibility warning from Holidays library v0.58
to inform you about an upcoming change in our API versioning strategy that may affect your
project's dependencies. Starting from version 1.0 onwards, we will be following a loose form of
Semantic Versioning (SemVer, https://semver.org) to provide clearer communication regarding any
potential breaking changes.

This means that while we strive to maintain backward compatibility, there might be occasional
updates that introduce breaking changes to our API. To ensure the stability of your projects,
we highly recommend pinning the version of our API that you rely on. You can pin your current
holidays v0.x dependency (e.g., holidays==0.58) or limit it (e.g., holidays<1.0) in order to
avoid potentially unwanted upgrade to the version 1.0 when it's released (ETA 2024Q4 - 2025Q1).

If you have any questions or concerns regarding this change, please don't 

In [10]:
print("Columns in df:", df.columns.tolist())

Columns in df: ['date', 'f_1__open']


In [11]:
# Remove timezone information from 'ds' column
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)


In [12]:
print(df['date'].dtype)


datetime64[us]


In [18]:
df.head()

,date,f_1__open
0,2024-10-15,163.0


In [13]:
prophet_df = df[['date', 'f_1__open']].rename(columns={'date': 'ds', 'f_1__open': 'y'})

In [15]:
prophet_df.head()

,ds,y
0,2024-10-15,163.21


In [17]:
# Predict using the model
predictions = model.predict(prophet_df)

# Display predictions
print(predictions)


          ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0 2024-10-15 -6.448571   78.774431    98.56846    -6.448571    -6.448571   

   additive_terms  additive_terms_lower  additive_terms_upper    daily  ...  \
0        95.16802              95.16802              95.16802  37.6362  ...   

      weekly  weekly_lower  weekly_upper     yearly  yearly_lower  \
0  99.716186     99.716186     99.716186 -42.184367    -42.184367   

   yearly_upper  multiplicative_terms  multiplicative_terms_lower  \
0    -42.184367                   0.0                         0.0   

   multiplicative_terms_upper       yhat  
0                         0.0  88.719449  

[1 rows x 22 columns]


In [18]:
predictions 

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-10-15,-6.448571,78.774431,98.56846,-6.448571,-6.448571,95.16802,95.16802,95.16802,37.6362,...,99.716186,99.716186,99.716186,-42.184367,-42.184367,-42.184367,0.0,0.0,0.0,88.719449


In [20]:
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [26]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 3,
    description = 'Predction of AMD close stock price',
    primary_key = ['y'],
    event_time = ['ds'],
    online_enabled = False,
)

In [27]:
results_fg.insert(prophet_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1283422


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/stock_prediction_results_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16e8926a8d0>, None)